# ETL Pipeline Preparation
Follow the instructions below to help you create your ETL pipeline.
### 1. Import libraries and load datasets.
- Import Python libraries
- Load `messages.csv` into a dataframe and inspect the first few lines.
- Load `categories.csv` into a dataframe and inspect the first few lines.

In [1]:
# import libraries
import pandas as pd
import numpy as np

In [57]:
# load messages dataset
messages = pd.read_csv("../data/disaster_messages.csv")
messages.head()

,id,message,original,genre
0,2,Weather update - a cold front from Cuba that c...,Un front froid se retrouve sur Cuba ce matin. ...,direct
1,7,Is the Hurricane over or is it not over,Cyclone nan fini osinon li pa fini,direct
2,8,Looking for someone but no name,"Patnm, di Maryani relem pou li banm nouvel li ...",direct
3,9,UN reports Leogane 80-90 destroyed. Only Hospi...,UN reports Leogane 80-90 destroyed. Only Hospi...,direct
4,12,"says: west side of Haiti, rest of the country ...",facade ouest d Haiti et le reste du pays aujou...,direct


In [30]:
# load categories dataset
categories = pd.read_csv("../data/disaster_categories.csv")
categories.head()

,id,categories
0,2,related-1;request-0;offer-0;aid_related-0;medi...
1,7,related-1;request-0;offer-0;aid_related-1;medi...
2,8,related-1;request-0;offer-0;aid_related-0;medi...
3,9,related-1;request-1;offer-0;aid_related-1;medi...
4,12,related-1;request-0;offer-0;aid_related-0;medi...


### 2. Merge datasets.
- Merge the messages and categories datasets using the common id
- Assign this combined dataset to `df`, which will be cleaned in the following steps

In [31]:
# merge datasets
df = pd.merge(messages, categories, on="id")
df.head()

,id,message,original,genre,categories
0,2,Weather update - a cold front from Cuba that c...,Un front froid se retrouve sur Cuba ce matin. ...,direct,related-1;request-0;offer-0;aid_related-0;medi...
1,7,Is the Hurricane over or is it not over,Cyclone nan fini osinon li pa fini,direct,related-1;request-0;offer-0;aid_related-1;medi...
2,8,Looking for someone but no name,"Patnm, di Maryani relem pou li banm nouvel li ...",direct,related-1;request-0;offer-0;aid_related-0;medi...
3,9,UN reports Leogane 80-90 destroyed. Only Hospi...,UN reports Leogane 80-90 destroyed. Only Hospi...,direct,related-1;request-1;offer-0;aid_related-1;medi...
4,12,"says: west side of Haiti, rest of the country ...",facade ouest d Haiti et le reste du pays aujou...,direct,related-1;request-0;offer-0;aid_related-0;medi...


### 3. Split `categories` into separate category columns.
- Split the values in the `categories` column on the `;` character so that each value becomes a separate column. You'll find [this method](https://pandas.pydata.org/pandas-docs/version/0.23/generated/pandas.Series.str.split.html) very helpful! Make sure to set `expand=True`.
- Use the first row of categories dataframe to create column names for the categories data.
- Rename columns of `categories` with new column names.
- Update the total value of the items to be 0 or 1

In [33]:
# create a dataframe of the 36 individual category columns
df_tmp = df['categories'].str.split(";", expand=True)
cols = [w.split("-")[0] for w in df_tmp.iloc[0, :].tolist()]
df_tmp.columns = cols
df_tmp = df_tmp.apply(lambda x: x.str.replace(".*?-([0-9]+)", r"\g<1>").astype(int)).clip(0, 1)
df = pd.concat([df.iloc[:, :-1], df_tmp], axis=1)

related                   1
request                   1
offer                     1
aid_related               1
medical_help              1
medical_products          1
search_and_rescue         1
security                  1
military                  1
child_alone               0
water                     1
food                      1
shelter                   1
clothing                  1
money                     1
missing_people            1
refugees                  1
death                     1
other_aid                 1
infrastructure_related    1
transport                 1
buildings                 1
electricity               1
tools                     1
hospitals                 1
shops                     1
aid_centers               1
other_infrastructure      1
weather_related           1
floods                    1
storm                     1
fire                      1
earthquake                1
cold                      1
other_weather             1
direct_report       

### 4. Remove duplicates.
- Check how many duplicates are in this dataset.
- Drop the duplicates.
- Confirm duplicates were removed.

In [35]:
# check number of duplicates
df_dub = df[df.duplicated(subset=None)]
print("Duplicates: {}".format(df_dub.shape[0]))
df_dub.head()

Duplicates: 171


,id,message,original,genre,related,request,offer,aid_related,medical_help,medical_products,...,aid_centers,other_infrastructure,weather_related,floods,storm,fire,earthquake,cold,other_weather,direct_report
164,202,?? port au prince ?? and food. they need gover...,p bay pap la syen ak manje. Yo bezwen ed gouve...,direct,1,1,0,1,0,0,...,0,0,0,0,0,0,0,0,0,0
165,202,?? port au prince ?? and food. they need gover...,p bay pap la syen ak manje. Yo bezwen ed gouve...,direct,1,1,0,1,0,0,...,0,0,0,0,0,0,0,0,0,0
658,804,elle est vraiment malade et a besoin d'aide. u...,she is really sick she need your help. please ...,direct,1,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
659,804,elle est vraiment malade et a besoin d'aide. u...,she is really sick she need your help. please ...,direct,1,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
660,804,elle est vraiment malade et a besoin d'aide. u...,she is really sick she need your help. please ...,direct,1,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0


In [36]:
# drop duplicates
df = df.drop_duplicates()

In [37]:
# check number of duplicates
df[df.duplicated(subset=None)]

,id,message,original,genre,related,request,offer,aid_related,medical_help,medical_products,...,aid_centers,other_infrastructure,weather_related,floods,storm,fire,earthquake,cold,other_weather,direct_report


Check for items with a very low mean

In [51]:
desc = df.describe()
desc[desc.columns[desc.loc['std', :] < 0.01]]

""
count
mean
std
min
25%
50%
75%
max


In [49]:
# since child_alone holds no relevant information, remove:
df = df.drop('child_alone', axis=1)

### 5. Save the clean dataset into an sqlite database.
You can do this with pandas [`to_sql` method](https://pandas.pydata.org/pandas-docs/stable/generated/pandas.DataFrame.to_sql.html) combined with the SQLAlchemy library. Remember to import SQLAlchemy's `create_engine` in the first cell of this notebook to use it below.

In [53]:
from sqlalchemy import create_engine
engine = create_engine('sqlite:///../data/disaster_data.db')
df.to_sql('texts', engine, index=False)

### 6. Use this notebook to complete `etl_pipeline.py`
Use the template file attached in the Resources folder to write a script that runs the steps above to create a database based on new datasets specified by the user. Alternatively, you can complete `etl_pipeline.py` in the classroom on the `Project Workspace IDE` coming later.

In [3]:
file = '../models/glove.twitter.27B.25d.txt'
def get_coefs(word, *arr): 
    return word, np.asarray(arr, dtype='float32')

embeddings_index = dict(get_coefs(*o.strip().split()) for o in open(file, encoding='utf-8'))


In [ ]:
i = 0
for ind in embeddings_index:
    if i > 10: break
    print("{}: {}".format(ind, embeddings_index.get(ind)))
    i += 1

In [ ]:
type(embeddings_index)

In [2]:
i = 0
def get_coefs(word, *arr): 
    return word, np.asarray(arr, dtype='float32')
for o in open('../models/glove.twitter.27B.25d.txt', encoding='utf-8'):
    if i > 10: break
    print(get_coefs(*o.strip().split()))
    i += 1

('<user>', array([ 0.62415 ,  0.62476 , -0.082335,  0.20101 , -0.13741 , -0.11431 ,
        0.77909 ,  2.6356  , -0.46351 ,  0.57465 , -0.024888, -0.015466,
       -2.9696  , -0.49876 ,  0.095034, -0.94879 , -0.017336, -0.86349 ,
       -1.3348  ,  0.046811,  0.36999 , -0.57663 , -0.48469 ,  0.40078 ,
        0.75345 ], dtype=float32))
('.', array([ 0.69586 , -1.1469  , -0.41797 , -0.022311, -0.023801,  0.82358 ,
        1.2228  ,  1.741   , -0.90979 ,  1.3725  ,  0.1153  , -0.63906 ,
       -3.2252  ,  0.61269 ,  0.33544 , -0.57058 , -0.50861 , -0.16575 ,
       -0.98153 , -0.8213  ,  0.24333 , -0.14482 , -0.67877 ,  0.7061  ,
        0.40833 ], dtype=float32))
(':', array([ 1.1242  ,  0.054519, -0.037362,  0.10046 ,  0.11923 , -0.30009 ,
        1.0938  ,  2.537   , -0.072802,  1.0491  ,  1.0931  ,  0.066084,
       -2.7036  , -0.14391 , -0.22031 , -0.99347 , -0.65072 , -0.030948,
       -1.0817  , -0.64701 ,  0.32341 , -0.41612 , -0.5268  , -0.047166,
        0.71549 ], dtype=float3